In [1]:
import pandas as pd
from datetime import datetime

In [2]:
class Config_cmdline:
    def __init__(self,line):
        self.line = line
        self.name = self.line[:20].rstrip()
        self.value = self.line[37:47].rstrip()
        self.addr  = self.line[61:].rstrip()
        
        self.add_leading_0byte_if_r()  
        
    def add_leading_0byte_if_r(self):
        if self.name[0] == 'r' and len(self.addr)==2:
            self.addr='00'+self.addr #.append('00') #is trailing since has been reversed by data_to_list()
        
def create_command_dict(commands):
    line   = [x for x,line in enumerate(commands)]
    name   = [Config_cmdline(line).name  for line in commands]
    value  = [Config_cmdline(line).value for line in commands]
    addr   = [Config_cmdline(line).addr  for line in commands]
    return {'line':line,'name':name, 'value': value, 'address': addr}
    
def create_diff_df_with_pagecalls(df1,df2):
    def add_missing_pagecalls(df_1,df_2,df_delta):
        df_delta_with_pages=df_delta.loc[:,['line','name','value','address']]
        for row in df_delta.index:
            line,name,value,addr = df_delta.loc[row,['line','name','value','address']]
            is_PMB_address=len(addr)==2
            if is_PMB_address:
                df_closest_preceeding_pageline = df_2[df_2['name']=='PAGE'][df2['line']<line].tail(1)
                df_delta_with_pages=df_delta_with_pages.append(df_closest_preceeding_pageline)
        df_delta_with_pages=df_delta_with_pages.sort_values(by=['line'])
        return df_delta_with_pages
    df_diff = df1.merge(df2,how='outer',indicator=True).loc[lambda x : x['_merge']=='right_only']
    df_diff = add_missing_pagecalls(df1,df2,df_diff)
    return df_diff

def create_file_header(header):
    diff_header=header
    now=datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    diff_header[-1]=now
    header_contents=''
    for line in diff_header:
        header_contents=header_contents+f"{line}"+'\n'
    header_contents=header_contents+'\n'+crc+'\n'+'\n'
    return header_contents

def create_data_contents(delta):
    delta_list=delta[['name','value','address']].values.tolist()
    data_contents=''
    for line in delta_list:
        formatted_line = f"{line[0]:<19}{'0x'+line[1]:<22}"+'# 0x'+line[2]+'\n'
        data_contents = data_contents+formatted_line
    return data_contents

def create_file_contents(header_contents,data_contents):
    file_contents=header_contents+data_contents
    return file_contents

In [3]:
oldprojectfile = 'sample_files/ISL69269-0 0x60e.txt'
newprojectfile = 'sample_files/ISL69269-0 0x60f.txt'
outputfilename = 'diff_file_ef.txt'

In [4]:
with open(oldprojectfile) as file:
    file1_contents = file.read()

In [5]:
with open(newprojectfile) as file:
    file2_contents = file.read()

In [6]:
lines1        = file1_contents.split('\n')
lines2        = file2_contents.split('\n')

commands1     = lines1[9:-1]
commands2     = lines2[9:-1]

header       = lines2[:6]
crc          = lines2[7]

In [7]:
command1_dict = create_command_dict(commands1)
command2_dict = create_command_dict(commands2)

In [8]:
df1 = pd.DataFrame(command1_dict) 
df2 = pd.DataFrame(command2_dict) 

In [9]:
delta = create_diff_df_with_pagecalls(df1,df2)
delta

,line,name,value,address
1008,136,rGUI5,A1A08E17,003F
1009,559,svidIccMax0,00000096,E421


In [10]:
hdr=create_file_header(header)
cmds=create_data_contents(delta)
file_contents=create_file_contents(hdr,cmds)
with open(outputfilename, 'w') as diff_file:
    diff_file.write(file_contents)